<a href="https://colab.research.google.com/github/shiraeisenberg/cocktail-recipe-search-engine/blob/main/cocktail_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets rich faiss-gpu sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 12.2 MB/s 
     |████████████████████████████████| 237 kB 30.8 MB/s 
     |████████████████████████████████| 85.5 MB 145 kB/s 
     |████████████████████████████████| 85 kB 5.6 MB/s 
     |████████████████████████████████| 115 kB 63.7 MB/s 
     |████████████████████████████████| 95 kB 6.0 MB/s 
     |████████████████████████████████| 163 kB 75.9 MB/s 
     |████████████████████████████████| 212 kB 71.0 MB/s 
     |████████████████████████████████| 127 kB 66.1 MB/s 
     |████████████████████████████████| 51 kB 8.9 MB/s 
     |████████████████████████████████| 5.5 MB 62.8 MB/s 
     |████████████████████████████████| 1.3 MB 53.0 MB/s 
     |████████████████████████████████| 7.6 MB 42.5 MB/s 
     |████████████████████████████████| 115 kB 73.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.wh

In [2]:
from datasets import load_dataset
from rich import inspect
import datasets

dataset = load_dataset("erwanlc/cocktails_recipe")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/erwanlc___csv/erwanlc--cocktails_recipe-b6028830061b788d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'glass', 'garnish', 'recipe', 'ingredients'],
        num_rows: 6956
    })
})

In [4]:
import pandas
dataset.set_format("pandas")
df = dataset["train"]
df

Dataset({
    features: ['title', 'glass', 'garnish', 'recipe', 'ingredients'],
    num_rows: 6956
})

In [5]:
df[0]['recipe'].tolist()

['Cut the top off a small pineapple and carefully scoop out the flesh from the base to leave a shell with 12mm (½ inch) thick walls. Place the shell in a freezer to chill. Remove the hard core from the pineapple flesh and discard; roughly chop the remaining flesh, add other ingredients and BLEND with one 12oz scoop of crushed ice. Pour into the pineapple shell and serve with straws. (The flesh of one pineapple blended with the following ingredients will fill at least two shells).']

In [6]:
df = df[:5000]
from datasets import Dataset
rdf = Dataset.from_pandas(df)
df

,title,glass,garnish,recipe,ingredients
0,Abacaxi Ricaço,Pineapple shell (frozen) glass,Cut a straw sized hole in the top of the pinea...,Cut the top off a small pineapple and carefull...,"[['1 whole', 'Pineapple (fresh)'], ['9 cl', 'H..."
1,Abbey,Coupe glass,Orange zest twist,SHAKE all ingredients with ice and fine strain...,"[['4.5 cl', 'Rutte Dry Gin'], ['2.25 cl', 'Lil..."
2,A.B.C. Cocktail,Nick & Nora glass,Lemon zest twist & Luxardo Maraschino cherry,TEAR mint and place in shaker. Add other ingre...,"[['7 fresh', 'Mint leaves'], ['3 cl', 'Tawny p..."
3,Absinthe Cocktail,Coupe glass,Mint leaf,SHAKE all ingredients with ice and fine strain...,"[['3 cl', 'La Fée Parisienne absinthe'], ['7.5..."
4,Absinthe Frappé,Old-fashioned glass,Mint sprig,SHAKE all ingredients with ice and fine strain...,"[['4.5 cl', 'La Fée Parisienne absinthe'], ['1..."
...,...,...,...,...,...
4995,Ojo Negro,Tiki mug or collins,"Dehydrated lime slice wheel, mint sprigs bouqu...",SHAKE all ingredients with ice and strain into...,"[['5 cl', 'Black Tears cacao & coffee spiced r..."
4996,Longshoreman,Old-fashioned glass,Flamed orange zest twist,STIR all ingredients with ice and strain into ...,"[['4.5 cl', 'Straight rye whiskey (100 proof /..."
4997,Loaded Pistol,Old-fashioned glass,Grapefruit juice & sea salt rim,STIR all ingredients with ice and strain into ...,"[['4.5 cl', 'Del Maguey VIDA mezcal'], ['2.25 ..."
4998,Cin-Cyn,Old-fashioned glass,Orange zest twist,STIR all ingredients with ice and strain into ...,"[['3 cl', 'Rutte Dry Gin'], ['3 cl', 'Cynar or..."


In [7]:
def concatenate_text(examples):
  return {
      "text" : examples["title"]
      + "\n"
      + examples["glass"]
      + "\n"
      + examples["garnish"]
      + "\n"
      + str(examples["recipe"])
      + "\n"
      + str(examples["ingredients"])
      
  }

concat_df = rdf.map(concatenate_text)

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [12]:
concat_df['text'][0]

"Abacaxi Ricaço\nPineapple shell (frozen) glass\nCut a straw sized hole in the top of the pineapple shell & replace it as a lid\nCut the top off a small pineapple and carefully scoop out the flesh from the base to leave a shell with 12mm (½ inch) thick walls. Place the shell in a freezer to chill. Remove the hard core from the pineapple flesh and discard; roughly chop the remaining flesh, add other ingredients and BLEND with one 12oz scoop of crushed ice. Pour into the pineapple shell and serve with straws. (The flesh of one pineapple blended with the following ingredients will fill at least two shells).\n[['1 whole', 'Pineapple (fresh)'], ['9 cl', 'Havana Club 3 Year Old rum'], ['2.25 cl', 'Lime juice (freshly squeezed)'], ['1.5 cl', 'White caster sugar']]"

In [13]:
concat_df

Dataset({
    features: ['title', 'glass', 'garnish', 'recipe', 'ingredients', 'text'],
    num_rows: 5000
})

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from transformers import AutoTokenizer, TFAutoModel
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [15]:
def cls_pooling(model_output):
  return model_output.last_hidden_state[:,0]

def get_embeddings(text_list):
  encoded_input = tokenizer(
      text_list, padding=True, truncation=True, return_tensors="tf"
  )
  encoded_input = {k: v for k,v in encoded_input.items()}
  model_output = model(**encoded_input)
  return cls_pooling(model_output)

In [17]:
embedding = get_embeddings(concat_df["text"][0])
embedding.shape

TensorShape([1, 768])

In [18]:
embeddings_dataset = concat_df.map(
    lambda x : {"embeddings" : get_embeddings(x["text"]).numpy()[0]}
)

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [19]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/5 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'glass', 'garnish', 'recipe', 'ingredients', 'text', 'embeddings'],
    num_rows: 5000
})

In [20]:
question = "What's a cocktail with pineapple?"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [21]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [22]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores",ascending=False,inplace=True)

In [24]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.text}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.recipe}")
    print("=" * 50)
    print()

COMMENT: Voodoo I Do
Old-fashioned glass
Pineapple wedge & Luxardo Maraschino cherry
MUDDLE pineapple in base of shaker. Add other ingredients, SHAKE with ice and fine strain into ice-filled glass.
[['1 ring', 'Pineapple (fresh)'], ['6 cl', 'Patrón Reposado tequila'], ['0.75 cl', 'De Kuyper Triple Sec (40%)'], ['1.5 cl', 'De Kuyper Apricot Brandy liqueur'], ['1.5 cl', 'Lime juice (freshly squeezed)']]
SCORE: 20.256549835205078
TITLE: Voodoo I Do
URL: MUDDLE pineapple in base of shaker. Add other ingredients, SHAKE with ice and fine strain into ice-filled glass.

COMMENT: Pineapple & Cardamom 'Martini'
Martini glass
Pineapple wedge
MUDDLE cardamom in base of shaker. Add other ingredients, SHAKE with ice and fine strain into a chilled glass.
[['3 whole', 'Green cardamom pods'], ['6 cl', 'Ketel One Vodka'], ['4.5 cl', 'Pineapple juice (fresh pressed)'], ['0.75 cl', 'Sugar syrup (65.0°brix, 2 sugar to 1 water rich syrup)']]
SCORE: 20.013898849487305
TITLE: Pineapple & Cardamom 'Martini'
UR

In [25]:
def get_results(question):
  question_embedding = get_embeddings([question]).numpy()
  question_embedding.shape
  scores, samples = embeddings_dataset.get_nearest_examples(
      "embeddings", question_embedding, k = 5
  )
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores",ascending=False,inplace=True)
  
  for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.text}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.recipe}")
    print("=" * 50)
    print()

In [26]:
get_results("what is a cocktail for a fun night out?")

COMMENT: Raspberry 'Martini' (by Dick Bradsell)
Martini glass
Raspberries
MUDDLE raspberries in base of shaker. Add other ingredients, SHAKE with ice and fine strain into chilled glass.
[['7 fresh', 'Raspberries (fresh)'], ['6 cl', 'Rutte Dry Gin'], ['3 cl', 'Giffard Creme de Framboise'], ['2 dash', 'Orange Bitters by Angostura (optional)']]
SCORE: 22.603178024291992
TITLE: Raspberry 'Martini' (by Dick Bradsell)
URL: MUDDLE raspberries in base of shaker. Add other ingredients, SHAKE with ice and fine strain into chilled glass.

COMMENT: Moonlight Cocktail (Gaz Regan's)
Flute glass
Orange zest twist
SHAKE all ingredients with ice and fine strain into chilled glass.
[['4.5 cl', 'Rutte Dry Gin'], ['2.25 cl', 'De Kuyper Triple Sec (40%)'], ['1.5 cl', 'Crème de violette liqueur'], ['0.75 cl', 'Lime juice (freshly squeezed)'], ['0.75 cl', 'Lemon juice (freshly squeezed)']]
SCORE: 22.48417854309082
TITLE: Moonlight Cocktail (Gaz Regan's)
URL: SHAKE all ingredients with ice and fine strain int

In [27]:
get_results("what's a cocktail for a gloomy night in the winter?")

COMMENT: Moonlight Cocktail (Gaz Regan's)
Flute glass
Orange zest twist
SHAKE all ingredients with ice and fine strain into chilled glass.
[['4.5 cl', 'Rutte Dry Gin'], ['2.25 cl', 'De Kuyper Triple Sec (40%)'], ['1.5 cl', 'Crème de violette liqueur'], ['0.75 cl', 'Lime juice (freshly squeezed)'], ['0.75 cl', 'Lemon juice (freshly squeezed)']]
SCORE: 27.41656494140625
TITLE: Moonlight Cocktail (Gaz Regan's)
URL: SHAKE all ingredients with ice and fine strain into chilled glass.

COMMENT: Raspberry 'Martini' (by Dick Bradsell)
Martini glass
Raspberries
MUDDLE raspberries in base of shaker. Add other ingredients, SHAKE with ice and fine strain into chilled glass.
[['7 fresh', 'Raspberries (fresh)'], ['6 cl', 'Rutte Dry Gin'], ['3 cl', 'Giffard Creme de Framboise'], ['2 dash', 'Orange Bitters by Angostura (optional)']]
SCORE: 27.36587905883789
TITLE: Raspberry 'Martini' (by Dick Bradsell)
URL: MUDDLE raspberries in base of shaker. Add other ingredients, SHAKE with ice and fine strain into

In [28]:
get_results("what's a cocktail for a date night before sex?")

COMMENT: Flip Cocktail (generic recipe)
Sour or Martini/Coupette glass
Dust with grated nutmeg
SHAKE all ingredients with the ice and strain back into shaker. DRY SHAKE (without ice) and pour into chilled glass.
[['6 cl', 'Brandy, whisk(e)y, gin, rum etc.'], ['1.5 cl', 'Sugar syrup (65.0°brix, 2 sugar to 1 water rich syrup)'], ['1 fresh', 'Egg (white & yolk)']]
SCORE: 25.754634857177734
TITLE: Flip Cocktail (generic recipe)
URL: SHAKE all ingredients with the ice and strain back into shaker. DRY SHAKE (without ice) and pour into chilled glass.

COMMENT: Cosmopolitan (no added sugar* & low-calorie)
Martini glass
Orange zest twist (flamed)
SHAKE all ingredients with ice and fine strain into chilled glass. (Be sure to use naturally sweetened cranberry drink rather than "juice" sweetened with sugar.)
[['4.5 cl', 'Ketel One Citroen Vodka'], ['1.5 cl', 'Agave Sec liqueur'], ['3 cl', 'Cranberry juice (sweetened)'], ['0.75 cl', 'Lime juice (freshly squeezed)'], ['0.5 cl', 'Sugar-free sweetener